In [ ]:
!pip install selenium -q

## 자동차 세부정보 크롤링

In [123]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
brand_list = [
{"name": "현대", "id": 303},
{"name": "기아", "id": 307},
{"name": "제네시스", "id": 304},
{"name": "BMW", "id": 362},
{"name": "벤츠", "id": 349},
{"name": "KGM", "id": 326},
{"name": "르노코리아", "id": 321},
{"name": "테슬라", "id": 611},
{"name": "쉐보레", "id": 312},
{"name": "볼보", "id": 459}
]
# 1. Selenium WebDriver 실행
wd = webdriver.Chrome()

cars = []
cars_salses_list = []
#브랜드 종류에 따라 데이터 크롤링
for brand in brand_list:
    print(brand['id'])
    # 3. 브랜드별 판매량 관련 웹페이지 이동
    url = f'https://auto.danawa.com/auto/?Work=record&Tab=Grand&Brand={str(brand['id'])}&Month=2024-01-00&MonthTo=2024-12-00'
    wd.get(url)
    
    
    # 4. 웹페이지 로딩 대기
    try:
        element = WebDriverWait(wd, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div > table > tbody'))
        )
        print('페이지 이동')
    except Exception as e:
        print(e)
    
    
    # 5. 데이터 크롤링
    rows = element.find_elements(By.TAG_NAME, 'tr')
    
    for row in rows:
        class_attribute = row.get_attribute('class')
        if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                    continue  
        model_name = row.find_element(By.CSS_SELECTOR, 'td.title a').text.strip()
        print(model_name)
        # 판매량
        sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')
        # 이미지 추출
        car_image = row.find_element(By.CSS_SELECTOR, 'td.title a img')  # 이미지 태그로 변경
        image_url = car_image.get_attribute('src')  # src 속성 값 가져오기
        print(image_url)

        # car_info_page_button = row.find_element(By.CSS_SELECTOR, 'td.title a')
        # car_info_page_button.send_keys(Keys.CONTROL + Keys.RETURN)  # 윈도우, 리눅스
        
        
        # # 다음 페이지로 전환
        # wd.switch_to.window(wd.window_handles[1])
        # time.sleep(2)
        # # 상세 정보 크롤링
        # html = wd.page_source
        # response = BeautifulSoup(html, 'html.parser')
        # car_detail = response.select_one('#info_model > p')
        # car_price = response.select_one('#info_summary > div.estimate-result__wrap > div.scroll-y > div:nth-child(1) > div:nth-child(2) > div.right.eTextPriceInfo')
        
        # 데이터 저장
        car = {
            '브랜드': brand['name'],
            '모델명': model_name,
            '세부정보' : car_detail.get_text(strip=True) if car_detail else 'N/A',
            '가격': car_price.get_text(strip=True) if car_price else 'N/A',
            'image' : image_url
        }
        # car_sales = {
        #     '모델명': model_name,
        #     '브랜드명' : brand['name'],
        #     '판매량': int(sales),
        #     '판매월': 0 # 1년
        # }
        cars.append(car)
        # cars_salses_list.append(car_sales)
        print(car)
        # print(car_sales)
        # 상세 정보 탭 닫기
        # wd.close()
        
        # # 원래 탭으로 돌아가기
        # wd.switch_to.window(wd.window_handles[0])
    
    # 7. WebDriver 종료
    print(cars)
    # print(cars_salses_list)
    

303
페이지 이동
싼타페
https://autoimg.danawa.com/photo/4435/model_200.png
{'브랜드': '현대', '모델명': '싼타페', '세부정보': '2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ', '가격': '42,670,000', 'image': 'https://autoimg.danawa.com/photo/4435/model_200.png'}
그랜저
https://autoimg.danawa.com/photo/4188/model_200.png
{'브랜드': '현대', '모델명': '그랜저', '세부정보': '2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ', '가격': '42,670,000', 'image': 'https://autoimg.danawa.com/photo/4188/model_200.png'}
포터2
https://autoimg.danawa.com/photo/1901/model_200.png
{'브랜드': '현대', '모델명': '포터2', '세부정보': '2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ', '가격': '42,670,000', 'image': 'https://autoimg.danawa.com/photo/1901/model_200.png'}
쏘나타 디 엣지
https://autoimg.danawa.com/photo/4466/model_200.png
{'브랜드': '현대', '모델명': '쏘나타 디 엣지', '세부정보': '2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ', '가격': '42,670,000', 'image': 'https://autoimg.danawa.com/photo/4466/model_200.png'}
아반떼
https://autoimg.danawa.com/photo/4455/model_200.png
{'브랜드': '현대', '모델명': '아반떼', '세부정보':

## 데이터베이스 저장

In [105]:
import mysql.connector as connector
# 데이터 베이스 연결 정보
args = {
    'host':'localhost', 
    'user':'root', 
    'password': 'Rladnjsdn9911!', 
    'port': 3306,
}

# 접속 여부 확인
try:
    conn = connector.connect(**args)
    cursor = conn.cursor()

    #DB 생성
    cursor.execute('create database if not exists car_project')
    # car 테이블 생성
    query = '''
        create table if not exists car_project.car(
            id int auto_increment primary key,
            brand varchar(100),
            model varchar(100), 
            detail varchar(100),
            price varchar(100)
        )
    '''
    cursor.execute(query)
    # 데이터 삽입
    for s in cars:
        query = 'insert into car_project.car values(null, %s, %s, %s, %s)'
        cursor.execute(query, list(s.values()))
    conn.commit()
    # car_sales 테이블 생성
    query = '''
        create table if not exists car_project.car_sales(
            id int auto_increment primary key,
            model varchar(100), 
            brand_temp varchar(100),
            sales int,
            month int
        )
    '''
    cursor.execute(query)
    for s in cars_salses_list:
        print(s)
        query = 'insert into car_project.car_sales values(null, %s, %s, %s, %s)'
        print(query, list(s.values()))
        cursor.execute(query, list(s.values()))
    # 모든 작업이 끝나면 commit 실행
    conn.commit()
except Exception as e:
    print(f'error: {e}')
else:
    print('정상 처리 되었습니다.')
finally:
    # 사용이 종료되었을 경우
    if cursor: cursor.close()
    if conn: conn.close()
    

{'모델명': '싼타페', '브랜드명': '현대', '판매량': 77159, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['싼타페', '현대', 77159, 0]
{'모델명': '그랜저', '브랜드명': '현대', '판매량': 71656, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['그랜저', '현대', 71656, 0]
{'모델명': '포터2', '브랜드명': '현대', '판매량': 58739, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['포터2', '현대', 58739, 0]
{'모델명': '쏘나타 디 엣지', '브랜드명': '현대', '판매량': 57325, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['쏘나타 디 엣지', '현대', 57325, 0]
{'모델명': '아반떼', '브랜드명': '현대', '판매량': 55793, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['아반떼', '현대', 55793, 0]
{'모델명': '투싼', '브랜드명': '현대', '판매량': 55257, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['투싼', '현대', 55257, 0]
{'모델명': '스타리아', '브랜드명': '현대', '판매량': 41118, '판매월': 0}
insert into car_project.car_sales values(null, %s, %s, %s, %s) ['스타리아', '현대', 41118, 0]
{'모델명': '캐

In [108]:
import pandas as pd
car_df = pd.DataFrame(cars)
car_sales = pd.DataFrame(cars_salses_list)
car_df.head()

,브랜드,모델명,세부정보,가격
0,현대,싼타페,"2025.01. 출시·중형SUV·가솔린+전기·1,598cc·복합 15.5 ㎞/ℓ","38,700,000"
1,현대,그랜저,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000"
2,현대,포터2,"2023.11. 출시·소형트럭·LPG·2,469cc·복합 7.0 ㎞/ℓ","20,390,000"
3,현대,쏘나타 디 엣지,"2025.01. 출시·중형·가솔린+전기·1,999cc·복합 19.4 ㎞/ℓ","32,320,000"
4,현대,아반떼,"2025.01. 출시·준중형·가솔린+전기·1,580cc·복합 21.1 ㎞/ℓ","24,830,000"


In [115]:
car_df.to_csv('D:\\sknai_project\\car.csv', index = False, encoding = 'euc-kr')

In [ ]:
# euc-kr, cp949 , utf-8

In [111]:
car_sales.to_csv('D:\\sknai_project\\car_sales.csv')

In [127]:
a.to_csv('D:\\sknai_project\\car_temp.csv', index = False, encoding = 'euc-kr')

In [125]:
a = pd.DataFrame(cars)

In [126]:
a.head()

,브랜드,모델명,세부정보,가격,image
0,현대,싼타페,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000",https://autoimg.danawa.com/photo/4435/model_20...
1,현대,그랜저,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000",https://autoimg.danawa.com/photo/4188/model_20...
2,현대,포터2,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000",https://autoimg.danawa.com/photo/1901/model_20...
3,현대,쏘나타 디 엣지,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000",https://autoimg.danawa.com/photo/4466/model_20...
4,현대,아반떼,"2025.01. 출시·준대형·가솔린+전기·1,598cc·복합 18.0 ㎞/ℓ","42,670,000",https://autoimg.danawa.com/photo/4455/model_20...
